# Imports & get data

In [8]:
import sys
sys.path.insert(0, "../")

import pandas as pd 
import numpy as np


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml


data = pd.read_csv("../data/communities.data", header=None, na_values=["?"])
from urllib.request import urlopen
names = urlopen("http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.names")
columns = [line.split(b' ')[1].decode("utf-8") for line in names if line.startswith(b'@attribute')]
data.columns = columns
data = data.dropna(axis = 1)
data = data.iloc[:, 3:]

## Evaluation helper

In [9]:
def evaluate_methods(X_prop_train, y_prop_train, X_cal, y_cal, X_test, y_test):
    from sklearn.metrics import mean_absolute_error as reg_metric
    from sklearn.linear_model import  LinearRegression
   
    myids = we_group
    keep = len(myids)
    
    #####
    print('RESIDUALS DIRECT...')
    myids = np.argsort(np.abs(errors_array)[:,-1])[0:keep]
    
    res_model = xgb.XGBRegressor(n_estimators=nest)
    res_model.fit(X_prop_train[myids,:], y_prop_train[myids])

    #####
    print('INTERVALS DIRECT...')
    myids = np.argsort(np.abs(interval_array)[:,-1])[0:keep]
    
    interval_model = xgb.XGBRegressor(n_estimators=nest)
    interval_model.fit(X_prop_train[myids,:], y_prop_train[myids])

    ####
    print('Actual eval fit...')
    cal_model = xgb.XGBRegressor(n_estimators=nest)
    cal_model.fit(X_cal, y_cal)

    ####
    print('MIX fit...')
    mix_model = xgb.XGBRegressor(n_estimators=nest)
    mix_model.fit(np.vstack((X_prop_train, X_cal)), np.hstack((y_prop_train, y_cal)))

 
    ####
    print('Plain...')
    plain_model = xgb.XGBRegressor(n_estimators=nest)
    plain_model.fit(X_prop_train, y_prop_train)

  
    ######
    print('NGBOOST...')
    from ngboost import NGBRegressor
    from ngboost.distns import Exponential, Normal, LogNormal

    base_learner = LinearRegression()

    learner_prop = NGBRegressor(Dist=Normal,
                                    Base=base_learner)
    learner_prop.fit(X_cal, y_cal)

    y_dists = learner_prop.pred_dist(X_prop_train)

    uncerts = y_dists[0:].params['scale']

    myids = np.argsort(uncerts)[0:keep]

    ngboost_model = xgb.XGBRegressor(n_estimators=nest)
    ngboost_model.fit(X_prop_train[myids,:], y_prop_train[myids])


    ########
    print('Bayes ridge...')
    from sklearn.linear_model import BayesianRidge

    learner_prop = BayesianRidge()

    learner_prop.fit(X_cal, y_cal)

    y_dists, uncerts = learner_prop.predict(X_prop_train, return_std=True)

    myids = np.argsort(uncerts)[0:keep]

    ridge_model = xgb.XGBRegressor(n_estimators=nest)
    ridge_model.fit(X_prop_train[myids,:], y_prop_train[myids])
    
    ########
    
    # import torch

    # from uq360.algorithms.variational_bayesian_neural_networks.bnn import BnnRegression

    # config = {
    #               "ip_dim":X_prop_train.shape[1], 
    #               "op_dim":1,                            
    #               "num_nodes":8, 
    #               "num_layers":5,
    #               "step_size":3e-2,
    #               "num_epochs":20,
    #           }

    # learner_prop = BnnRegression(config = config, prior = 'Gaussian') #, Hshoe, RegHshoe

    # learner_prop.fit(torch.Tensor(X_cal), torch.Tensor(y_cal))

    # y_dists, lower, upper, _, _ = learner_prop.predict(torch.Tensor(X_prop_train))

    # uncerts = upper-lower


    # myids = np.argsort(uncerts)[0:keep]

    # bnn_model = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    # bnn_model.fit(X_prop_train[myids,:], y_prop_train[myids])


    ########
    
    from sklearn.gaussian_process import GaussianProcessRegressor

    gpr = GaussianProcessRegressor(random_state=seed)
    gpr.fit(X_cal, y_cal)

    y_dists, uncerts = gpr.predict(X_prop_train, return_std=True)

    myids = np.argsort(uncerts)[0:keep]

    gp_model = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    gp_model.fit(X_prop_train[myids,:], y_prop_train[myids])

    # return res_model, interval_model, cal_model, mix_model, plain_model, ngboost_model, ridge_model, bnn_model, gp_model
    return res_model, interval_model, cal_model, mix_model, plain_model, ngboost_model, ridge_model,  gp_model




### Process dataset

In [10]:
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.metrics import mean_absolute_error as reg_metric
import random
seed= 589

X = data.drop('ViolentCrimesPerPop', axis=1)
y = data.ViolentCrimesPerPop

cols = X.columns

random.seed(seed)

group = 'racepctblack'

protected = np.where(X[group] >= 0.5)[0]

test_ids_protected = random.sample(list(protected) , int(0.90*len(protected)))

priviledged = np.where(X[group] <= 0.5)[0]

test_ids_priviledged = random.sample(list(priviledged) , int(0.01*len(priviledged)))

test_ids = np.hstack((test_ids_priviledged, test_ids_protected))

test_ids = np.array(test_ids)

train_ids = np.setdiff1d(range(len(y)), test_ids, assume_unique=True)

X_prop_train, y_prop_train = X.iloc[train_ids,:], y.iloc[train_ids]

X_eval, y_eval = X.iloc[test_ids,:], y.iloc[test_ids]

X_test, X_cal, y_test, y_cal = train_test_split(X_eval, y_eval,
                                        test_size=0.15, random_state=seed)

X_prop_train, X_cal, X_test = np.array(X_prop_train), np.array(X_cal), np.array(X_test)
y_prop_train, y_cal, y_test = np.array(y_prop_train), np.array(y_cal), np.array(y_test)



# Computation & sculpting

In [11]:
import numpy as np
from sklearn import datasets
from fairlearn.reductions import ExponentiatedGradient
from fairlearn.reductions import GridSearch 
from fairlearn.reductions import BoundedGroupLoss, ZeroOneLoss
from sklearn.metrics import mean_absolute_error
from fairlearn.metrics import MetricFrame

from copy import deepcopy
import random
from triage.triage import Triage
from tqdm import tqdm
import xgboost as xgb

final_results = []

nest = 50
learner = xgb.XGBRegressor(n_estimators=nest)
learner.fit(X_prop_train, y_prop_train)


triage_array = None
interval_array = None
crps_array = None
cpds_array = None
errors_array = None

test=False


y_eval = y_prop_train 
X_eval = X_prop_train



triage = Triage(X_eval = X_eval, y_eval = y_eval, X_cal=X_cal, y_cal=y_cal, nest=nest, learner=learner)
groups_ids, raw_metrics = triage.run()
    
triage_array = raw_metrics['score_metric']
errors_array = raw_metrics['errors_array']
interval_array = raw_metrics['interval_array']
crps_array = raw_metrics['crps_array']
cpds_array = raw_metrics['cpds_array']
preds_array = raw_metrics['preds_array']
upper_array = raw_metrics['upper_array']
lower_array = raw_metrics['lower_array']

metric = triage_array  
percentile_thresh = 75
thresh = 0.33
conf_thresh_low = thresh
conf_thresh_high = 1 - thresh
conf_thresh = 0.5

uncert= np.mean(metric * (1 - metric), axis=-1)
confidence = np.mean(metric, axis=-1)

# Get groups and mainly well-estimated groups
oe_group = np.where(
    (confidence <= conf_thresh_low)
    & (uncert<= np.percentile(uncert, percentile_thresh))
)[0]
ue_group = np.where(
    (confidence >= conf_thresh_high)
    & (uncert<= np.percentile(uncert, percentile_thresh))
)[0]

combined_group = np.concatenate((oe_group, ue_group))

we_group = []
for id in range(len(confidence)):
    if id not in combined_group:
        we_group.append(id)

we_group = np.array(we_group)

groups = []
for i in range(len(triage_array)):
    if i in oe_group:
        groups.append(0)
    if i in we_group:
        groups.append(1)
    if i in ue_group:
        groups.append(2)
        


bgl = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)
exp_model = ExponentiatedGradient(xgb.XGBRegressor(n_estimators=nest),
                   constraints=bgl)
exp_model.fit(X_prop_train, y_prop_train, sensitive_features=(y_prop_train >= 0.5).astype(int))


bgl = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)

sweep = GridSearch(xgb.XGBRegressor(n_estimators=nest),
                   constraints=bgl,
                   grid_size=3)

sweep.fit(X_prop_train, y_prop_train, sensitive_features=(y_prop_train >= 0.5).astype(int))

fair0_model = sweep.predictors_[0]
fair1_model = sweep.predictors_[1]
fair2_model = sweep.predictors_[2]

myids = we_group
keep = len(myids)
learner_prop_triage = xgb.XGBRegressor(n_estimators=nest)
learner_prop_triage.fit(X_prop_train[myids,:], y_prop_train[myids])

# uncomment to run BNN
#res_model, interval_model, cal_model, mix_model, plain_model, ngboost_model, ridge_model, bnn_model, gp_model = evaluate_methods(X_prop_train, y_prop_train, X_cal, y_cal, X_test, y_test)
res_model, interval_model, cal_model, mix_model, plain_model, ngboost_model, ridge_model, gp_model = evaluate_methods(X_prop_train, y_prop_train, X_cal, y_cal, X_test, y_test)


RESIDUALS DIRECT...
INTERVALS DIRECT...
Actual eval fit...
MIX fit...
Plain...
NGBOOST...
[iter 0] loss=0.1805 val_loss=0.0000 scale=2.0000 norm=0.9863
[iter 100] loss=-1.0216 val_loss=0.0000 scale=2.0000 norm=0.9479
[iter 200] loss=-2.1329 val_loss=0.0000 scale=4.0000 norm=1.9894
[iter 300] loss=-4.1329 val_loss=0.0000 scale=4.0000 norm=1.9998
[iter 400] loss=-7.9329 val_loss=0.0000 scale=8.0000 norm=4.0000
Bayes ridge alea...
Bayes ridge...


# Regression measures for Fairness

In [12]:
def calculate_regression_measures(y, y_hat, protected, privileged):

    from sklearn.linear_model import LogisticRegression

    unique_protected = np.unique(protected)
    unique_unprivileged = unique_protected[unique_protected != privileged]

    data = pd.DataFrame(columns=['subgroup', 'independence', 'separation', 'sufficiency'])

    for unprivileged in unique_unprivileged:
        # filter elements
        array_elements = np.isin(protected, [privileged, unprivileged])

        y_u = ((y[array_elements] - y[array_elements].mean()) / y[array_elements].std()).reshape(-1, 1)
        s_u = ((y_hat[array_elements] - y_hat[array_elements].mean()) / y_hat[array_elements].std()).reshape(-1, 1)

        a = np.where(protected[array_elements] == privileged, 1, 0)

        p_s = LogisticRegression()
        p_ys = LogisticRegression()
        p_y = LogisticRegression()

        p_s.fit(s_u, a)
        p_y.fit(y_u, a)
        p_ys.fit(np.c_[y_u, s_u], a)

        pred_p_s = p_s.predict_proba(s_u.reshape(-1, 1))[:, 1]
        pred_p_y = p_y.predict_proba(y_u.reshape(-1, 1))[:, 1]
        pred_p_ys = p_ys.predict_proba(np.c_[y_u, s_u])[:, 1]

        n = len(a)

        r_ind = ((n - a.sum()) / a.sum()) * (pred_p_s / (1 - pred_p_s)).mean()
        r_sep = ((pred_p_ys / (1 - pred_p_ys) * (1 - pred_p_y) / pred_p_y)).mean()
        r_suf = ((pred_p_ys / (1 - pred_p_ys)) * ((1 - pred_p_s) / pred_p_s)).mean()
        
        scores = 0
        if r_ind>1.25:
            scores+=1
        if r_sep>1.25:
            scores+=1
        if r_suf>1.25:
            scores+=1
            
        from sklearn.metrics import mean_squared_error
            
        mae =  reg_metric(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)

        to_append = {'independence': r_ind,
                      'separation': r_sep,
                      'sufficiency': r_suf,
                      'scores': scores,
                      'mae': mae,
                      'mse': mse,
                      'fair': scores<1}
    return to_append


# Compute results

In [13]:
X_test_pd = pd.DataFrame(X_test, columns=cols)
protected = np.where(X_test_pd[group] >= 0.5, 'majority_black', "else")
privileged = 'else'

metrics = {}

y_pred = learner.predict(X_test)
metrics['base'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = learner_prop_triage.predict(X_test)
metrics['triage'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = fair0_model.predict(X_test)
metrics['fair0_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = fair1_model.predict(X_test)
metrics['fair1_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = fair2_model.predict(X_test)
metrics['fair2_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = exp_model.predict(X_test)
metrics['fair_exp'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = res_model.predict(X_test)
metrics['res_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = interval_model.predict(X_test)
metrics['interval_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = cal_model.predict(X_test)
metrics['cal_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = mix_model.predict(X_test)
metrics['mix_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = plain_model.predict(X_test)
metrics['plain_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = ngboost_model.predict(X_test)
metrics['ngboost_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = ridge_model.predict(X_test)
metrics['ridge_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

# y_pred = bnn_model.predict(X_test)
# metrics['bnn_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)

y_pred = gp_model.predict(X_test)
metrics['gp_model'] = calculate_regression_measures(y=y_test, y_hat=y_pred, protected=protected, privileged=privileged,)


In [14]:
my_df = pd.DataFrame.from_dict(metrics)

my_df = my_df.T

my_df

,independence,separation,sufficiency,scores,mae,mse,fair
base,1.291645,1.178324,1.009437,1,0.18411,0.055423,False
triage,1.209842,1.112505,1.003123,0,0.177235,0.052826,True
fair0_model,1.010078,0.998461,1.017691,0,0.295557,0.125423,True
fair1_model,1.214557,1.117667,0.99944,0,0.19306,0.060235,True
fair2_model,1.307011,1.164173,0.998121,1,0.255597,0.110098,False
fair_exp,1.291645,1.178324,1.009437,1,0.18411,0.055423,False
res_model,1.405469,1.194699,1.034667,1,0.174659,0.050284,False
interval_model,1.258044,1.147145,1.0143,1,0.171621,0.046789,False
cal_model,1.141745,1.08543,0.993856,0,0.217411,0.068866,True
mix_model,1.268938,1.132202,1.021178,1,0.170751,0.046694,False
